In [167]:
import json
import requests
from bs4 import BeautifulSoup as bs
import random
import math

def currentAddy():
    data = json.loads(requests.get('http://ip-api.com/json/').content)
    lat = data['lat']
    lon = data['lon']
    print('User location: ' + str(lat) + ', ' + str(lon))
    result = requests.get(('http://www.mapquestapi.com/geocoding/v1/reverse?'
                         'key=HHtMzHkqE6RLT3EA3RA1TuZ8G2rVyvt4&location=' + str(lat) +',' + str(lon)))
    
    rJson = json.loads(result.content)
    location = rJson['results'][0]['locations'][0]
    return (location['street'] + ' ' + location['adminArea5'] + ' ' 
           + location['adminArea3'] + ' ' + location['postalCode'])
    
currentAddy()

User location: 38.9091, -77.0203


'1448 6th St NW Washington DC 20001'

In [159]:
def optimalRoute(fromLoc, toLoc):
    apiKey = 'HHtMzHkqE6RLT3EA3RA1TuZ8G2rVyvt4'
    requestStart = ("http://www.mapquestapi.com/directions/v2/route?" 
                      "key=HHtMzHkqE6RLT3EA3RA1TuZ8G2rVyvt4&from=")

    requestFrom = fromLoc.replace(' ', '+')

    requestTo = ("&to=" + toLoc.replace(' ' , '+'))
    #print(requestStart + requestFrom + requestTo)

    output = requests.get("http://www.mapquestapi.com/directions/v2/route?" 
                          "key=HHtMzHkqE6RLT3EA3RA1TuZ8G2rVyvt4&from=" 
                          + requestFrom + requestTo)
    
    jOutput = json.loads(output.content)
    #print(jOutput)
    distance = jOutput["route"]['distance']
    driveTime = jOutput['route']['time']
    return [distance, driveTime]

In [170]:
sampleAddress = currentAddy()

#sampleAddress = '11408 Cherry Hill rd, Silver Spring, MD 20705'
searchRad = random.choice([5, 10, 15, 20, 25, 30])
print('Search Radius: ' + str(searchRad))
with open('hospital-ledger.json') as f:
    ledger = json.load(f)
_zip = sampleAddress[-5:]

resultsPage = requests.get('https://www.zip-codes.com/zip-code-radius-finder.asp?zipmileslow=0&zipmileshigh=' + 
                           str(searchRad) + '&zip1=' + _zip + '&submit=Search')
page = bs(resultsPage.content, 'html.parser')
resultsTable = page.findAll('table')[0].findAll('table')[1].findAll('tr')
#print(resultsTable)
statesList = []
zipList = []
#print(resultsTable[1])
for result in resultsTable:
    sList = result.findAll('td')
    if len(sList) is 8:
        _state = sList[4].get_text()
        _zip = sList[1].get_text()
        if state not in statesList:
            statesList.append(_state)
        if _zip not in zipList:
            zipList.append(_zip)
            
zipList.append('21229')
fetchedIDs = []            
hospitalNames = []      
hWaits = []
hAddies = []

for state in statesList:
    print('ledger searching state: ' + state)
    if _state in ledger['State']:
        for _zip in zipList:
            if _zip in ledger['State'][state]['Zip']:
                #print('Zip code: ' + _zip)
                _hospDir = ledger['State'][state]['Zip'][_zip]['Hospital']
                for idx, _hosp in enumerate(_hospDir):
                    _hName = list(_hospDir)[idx]
                    if _hName not in hospitalNames:
                        hospitalNames.append(_hName)
                        fetchedIDs.append(_hospDir[_hName]['waitID'])
                        _address = _hospDir[_hName]['address'] + ' ' + _state + ' ' + _zip
                        hAddies.append(_address) 
                        print('Hospital within proximity: ' + _hName + ' waitID: ' + 
                              str(_hospDir[_hName]['waitID']), end ='\n')
                        print('Hospital address: ' + _address)
                        _calcOut = optimalRoute(sampleAddress, _address)
                        print('Distance to hospital: ' + 
                              str(_calcOut[0]) 
                              + ' miles, drive time: ' + str(math.trunc(_calcOut[1] / 60)) + ' minutes', end='\n'*2)
    
#print(_zip)
#print(resultsTable[1].findAll('td')[4].get_text())
#print(statesList)
#print(zipList)
#print(_table)
#print(ledger)

User location: 38.9091, -77.0203
Search Radius: 25
ledger searching state: DC
Hospital within proximity: MedStar Health Urgent Care - Adams Morgan waitID: 1276
Hospital address: 1805 Columbia Rd. NW MD 20009
Distance to hospital: 2.135 miles, drive time: 6 minutes

Hospital within proximity: Metro Immediate and Primary Care - Capitol Hill/Noma waitID: 332
Hospital address: 220 L St NE  MD 20002
Distance to hospital: 3.932 miles, drive time: 9 minutes

Hospital within proximity: MedStar Health Urgent Care - Capitol Hill waitID: 1278
Hospital address: 228 7th St. SE MD 20003
Distance to hospital: 2.802 miles, drive time: 8 minutes

Hospital within proximity: Metro Immediate and Primary Care - Cleveland Park waitID: 331
Hospital address: 2902 Porter Street, NW MD 20008
Distance to hospital: 3.732 miles, drive time: 10 minutes



In [99]:
import urllib, json

data = json.loads(requests.get('http://ip-api.com/json/').content)
print(str(data["lat"]) + ', ' + str(data['lon']))

38.9091, -77.0203
